In [0]:
# MSE function
import cv2
from sklearn.metrics import mean_squared_error

orig_img = cv2.imread('hara_original.png', 0)

def mse_function(imageA, imageB):
  ms_err = mean_squared_error(imageA, imageB)
  print(ms_err)
  return ms_err

In [0]:
# Inverse filtering

In [0]:
import numpy as np
import cv2
from PIL import Image
import scipy.fftpack as fp

image_filename = 'hara_cylind_blur'
image_ext = '.png'

img = cv2.imread(str(image_filename) +  str(image_ext), 0)

In [0]:
import numpy as np
import cv2

F1 = np.zeros(img.shape)
F2 = np.zeros(img.shape)
iT = 10
N = 498

def inverse_filter(img):
  dft = np.fft.fft(img)
  img_real = dft.real
  img_imag = dft.imag

  for n in range(0, img.shape[0]):
    for m in range(0, img.shape[1]):
      out_denom = np.sin(iT * np.pi * m / N)
      if(out_denom < 0.07 and out_denom > -0.07):
        F1[n][m] = img_real[n][m]
        F2[n][m] = img_imag[n][m]
      else:
        F1[n][m] = (iT * np.sin(np.pi * m / N) 
        * (img_real[n][m] * np.cos(np.pi * m * (iT - 1) / N) - img_imag[n][m] * np.sin(np.pi * m * (iT - 1) / N)) 
        / out_denom)
        F2[n][m] = (iT * np.sin(np.pi * m / N) 
        * (img_real[n][m] * np.sin(np.pi * m * (iT - 1) / N) + img_imag[n][m] * np.cos(np.pi * m * (iT - 1) / N)) 
        / out_denom)
  return F1, F2

In [0]:
proc_real, proc_imag = inverse_filter(img)
dft_mat = proc_real + 1j*proc_imag
inv_proc = np.fft.ifft(dft_mat)

In [0]:
inv_real = inv_proc.real
output_name = str(image_filename) + "_inverse" +  str(image_ext)
cv2.imwrite(output_name, inv_real)

True

In [0]:
cylind_blur = cv2.imread('hara_cylind_blur_inverse.png', 0)
cylind_gauss = cv2.imread('hara_cylind_gauss_inverse.png', 0)
print("Cylindrical blur MSE: " + str(mse_function(cylind_blur, orig_img)))
print("Cylindrical Gauss MSE: " + str(mse_function(cylind_gauss, orig_img)))

83.51753498368473
Cylindrical blur MSE: 83.51753498368473
93.77850856551206
Cylindrical Gauss MSE: 93.77850856551206


In [0]:
# Wiener filtering

In [0]:
import numpy as np
import cv2
from PIL import Image
import scipy.fftpack as fp

image_filename = 'hara_real_gauss'
image_ext = '.png' 
img = cv2.imread(str(image_filename) +  str(image_ext), 0)

In [0]:
import numpy as np
import cv2

F1 = np.zeros(img.shape)
F2 = np.zeros(img.shape)
iT = 10
N = 498
gamma = 0.06

def wiener_filter(img):
  dft = np.fft.fft(img)
  img_real = dft.real
  img_imag = dft.imag

  for n in range(0, img.shape[0]):
    for m in range(0, img.shape[1]):      
      if(m == 0):
        F1[n][m] = img_real[n][m] / (1 + gamma)
        F2[n][m] = img_imag[n][m] / (1 + gamma)
      else:
        F1[n][m] = ((iT * np.sin(np.pi * m / N) * np.sin(iT * np.pi * m / N))
          / (np.power((np.sin(iT * np.pi * m / N)), 2) + (gamma * np.power(iT, 2) * np.power((np.sin(np.pi * m / N)), 2)) )
          * (img_real[n][m] * np.cos(np.pi * m * (iT - 1) / N) - img_imag[n][m] * np.sin(np.pi * m * (iT - 1) / N)))
        F2[n][m] = ((iT * np.sin(np.pi * m / N) * np.sin(iT * np.pi * m / N)) 
          / (np.power((np.sin(iT * np.pi * m / N)), 2) + (gamma * np.power(iT, 2) * np.power((np.sin(np.pi * m / N)), 2)))
          * (img_real[n][m] * np.sin(np.pi * m * (iT - 1) / N) + img_imag[n][m] * np.cos(np.pi * m * (iT - 1) / N)))
  return F1, F2

In [0]:
proc_real, proc_imag = wiener_filter(img)
dft_mat = proc_real + 1j*proc_imag
inv_proc = np.fft.ifft(dft_mat)

In [0]:
inv_real = inv_proc.real
output_name = str(image_filename) + "_wiener" +  str(image_ext)
cv2.imwrite(output_name, inv_real)

True

In [0]:
cylind_gauss = cv2.imread('hara_cylind_gauss_wiener.png', 0)
print("Cylindrical Gauss MSE: " + str(mse_function(cylind_gauss, orig_img)))
cylind_gauss = cv2.imread('hara_cylind_blur_wiener.png', 0)
print("Cylindrical Blur MSE: " + str(mse_function(cylind_blur, orig_img)))

95.14779038027108
Cylindrical Gauss MSE: 95.14779038027108
83.51753498368473
Cylindrical Blur MSE: 83.51753498368473


In [0]:
# Constrained matrix inversion filtering

In [0]:
import numpy as np
import cv2
from PIL import Image
import scipy.fftpack as fp

image_filename = 'hara_real_gauss'
image_ext = '.png' 
img = cv2.imread(str(image_filename) +  str(image_ext), 0)

In [0]:
import numpy as np
import cv2

F1 = np.zeros(img.shape)
F2 = np.zeros(img.shape)
A = np.zeros(img.shape)
N = 498
gamma = 0.002

def constrained_filter(img):
  dft = np.fft.fft(img)
  img_real = dft.real
  img_imag = dft.imag

  for n in range(0, img.shape[0]):
    for m in range(0, img.shape[1]):      
      if(m == 0):
        A[n][m] = np.sqrt((np.power((N-5), 2) + (2 * (N-5) * np.cos(2 * np.pi * n / N)) + (2 * (N-5) * np.cos(2 * np.pi * (N-1) / N)) + (2  * np.cos(2 * np.pi * (N-2) * n / N))))
        F1[n][m] = img_real[n][m] / (1 + gamma * A[n][m])
        F2[n][m] = img_imag[n][m] / (1 + gamma * A[n][m])
      else:
        A[n][m] = np.sqrt((25 + 2 - (10 * np.cos(2 * np.pi * n / N)) - (10 * np.cos(2 * np.pi * (N-1) / N)) + (2  * np.cos(2 * np.pi * (N-2) * n / N))))
        F1[n][m] = ((iT * np.sin(np.pi * m / N) * np.sin(iT * np.pi * m / N))
          * (img_real[n][m] * np.cos(np.pi * m * (iT - 1) / N) - img_imag[n][m] * np.sin(np.pi * m * (iT - 1) / N))          
          / (np.power((np.sin(iT * np.pi * m / N)), 2) + (gamma * A[n][m] * np.power(iT, 2) * np.power((np.sin(np.pi * m / N)), 2)) ))
        F2[n][m] = ((iT * np.sin(np.pi * m / N) * np.sin(iT * np.pi * m / N))
          * (img_real[n][m] * np.sin(np.pi * m * (iT - 1) / N) + img_imag[n][m] * np.cos(np.pi * m * (iT - 1) / N))          
          / (np.power((np.sin(iT * np.pi * m / N)), 2) + (gamma * A[n][m] * np.power(iT, 2) * np.power((np.sin(np.pi * m / N)), 2)) ))
  return F1, F2

In [0]:
proc_real, proc_imag = constrained_filter(img)
dft_mat = proc_real + 1j*proc_imag
inv_proc = np.fft.ifft(dft_mat)

In [81]:
inv_real = inv_proc.real
output_name = str(image_filename) + "_constrained" +  str(image_ext)
cv2.imwrite(output_name, inv_real)

True

In [74]:
cylind_gauss = cv2.imread('hara_cylind_gauss_constrained.png', 0)
print("Cylindrical Gauss MSE: " + str(mse_function(cylind_gauss, orig_img)))
cylind_blur = cv2.imread('hara_cylind_blur_constrained.png', 0)
print("Cylindrical Blur MSE: " + str(mse_function(cylind_blur, orig_img)))

99.1017860504518
Cylindrical Gauss MSE: 99.1017860504518
103.70734892695783
Cylindrical Blur MSE: 103.70734892695783
